In [1]:
project: getenv `project_id
csbucketname: getenv `csbucketname

# From BigQuery to kdb

In [2]:
// extract from BigQuery to Cloud Storage
system "bq extract ", project, ":bqkdb.allBQSimpleTypes gs://", csbucketname, "/allBQSimpleTypes.csv"

Waiting on bqjob_r6df8bcd852d8a0b7_0000016e462d269e_1 ... (1s) Current status: DONE   

""


In [3]:
csfilename: "gs://", csbucketname, "/allBQSimpleTypes.csv"

In [4]:
// Copy from Cloud Storage to local box
system "gsutil cp ", csfilename, " /tmp/"

Copying gs://storagebodon/allBQSimpleTypes.csv...
- [1 files][  229.0 B/  229.0 B]                                                
Operation completed over 1 objects/229.0 B.                                      


## Setting types manually

* using q time results in losing microsecond precision
* postprocessing is needed for types 
    * BOOL
    * TIMESTAMP

In [5]:
allBQSimpleTypes: ("sIF**DTP"; enlist ",") 0: read0 hsym `$"/tmp/allBQSimpleTypes.csv"

In [6]:
\c 25 125

In [7]:
allBQSimpleTypes

s    int f     b       ts                               date       time         dt                           
-------------------------------------------------------------------------------------------------------------
AAPL 200 104.9 "false" "2019-11-07 14:02:43.013946 UTC" 2019.11.07 14:02:43.013 2019.11.07D14:02:43.013946000
GOOG 42  100.3 "true"  "2019-11-06 01:45:00 UTC"        2019.11.07 14:02:43.013 2019.11.07D14:02:43.013946000


Converting true/false literals to bool values

In [8]:
bigQueryToKdbBoolMap: ("true";"false")!10b

In [9]:
allBQSimpleTypes_fixed: update bigQueryToKdbBoolMap b from allBQSimpleTypes

Converting timestamps by chopping of " UTC" postfix.

In [10]:
update "P"$-4_/:ts from `allBQSimpleTypes_fixed

`allBQSimpleTypes_fixed


In [11]:
allBQSimpleTypes_fixed

s    int f     b ts                            date       time         dt                           
----------------------------------------------------------------------------------------------------
AAPL 200 104.9 0 2019.11.07D14:02:43.013946000 2019.11.07 14:02:43.013 2019.11.07D14:02:43.013946000
GOOG 42  100.3 1 2019.11.06D01:45:00.000000000 2019.11.07 14:02:43.013 2019.11.07D14:02:43.013946000


## Automatic type conversion

In [12]:
\l utils/csvutil.q

In [13]:
allBQSimpleTypes_auto: .csv.read hsym `$"/tmp/allBQSimpleTypes.csv"

In [14]:
allBQSimpleTypes_auto

s      int f     b       ts                               date       time                 dt                           
-----------------------------------------------------------------------------------------------------------------------
"AAPL" 200 104.9 "false" "2019-11-07 14:02:43.013946 UTC" 2019.11.07 0D14:02:43.013946000 2019.11.07D14:02:43.013946000
"GOOG" 42  100.3 "true"  "2019-11-06 01:45:00 UTC"        2019.11.07 0D14:02:43.013946000 2019.11.07D14:02:43.013946000


In [15]:
meta allBQSimpleTypes_auto

c   | t f a
----| -----
s   | C    
int | h    
f   | e    
b   | C    
ts  | C    
date| d    
time| n    
dt  | p    


Automatic type conversion works well for all types except for BOOL and TIMESTAMP.

# From kdb to BigQuery

Let us save the fixed kdb table to CSV

In [16]:
save `:/tmp/allBQSimpleTypes_fixed.csv

`:/tmp/allBQSimpleTypes_fixed.csv


In [17]:
system "bq load --autodetect bqkdb.allBQSimpleTypes_auto /tmp/allBQSimpleTypes_fixed.csv"

Waiting on bqjob_r7c47ac4145124624_0000016e462d6953_1 ... (1s) Current status: DONE   

""
""


We can see that boolean (b) and timestamp columns (ts, dt) are not casted properly, they are string columns.

In [18]:
system "bq show bqkdb.allBQSimpleTypes_auto"

"Table ferenc-world:bqkdb.allBQSimpleTypes_auto"
""
"   Last modified        Schema        Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Labe..
" ----------------- ----------------- ------------ ------------- ------------ ------------------- ------------------ -----..
"  07 Nov 15:03:36   |- s: string      2            216                                                                   ..
"                    |- int: integer                                                                                      ..
"                    |- f: float                                                                                          ..
"                    |- b: integer                                                                                        ..
"                    |- ts: string                                                                                        ..
"                    |- date: date                                       

We can convert bool and timestamp manually in q

In [19]:
kdbToBigQueryBoolMap: value[bigQueryToKdbBoolMap]!key bigQueryToKdbBoolMap

In [20]:
allBQSimpleTypes2: update kdbToBigQueryBoolMap b,
    @[; 4 7 10; :; "-- "] each string ts, 
    @[; 4 7 10; :; "-- "] each string dt from allBQSimpleTypes_fixed

In [21]:
allBQSimpleTypes2

s    int f     b       ts                              date       time         dt                             
--------------------------------------------------------------------------------------------------------------
AAPL 200 104.9 "false" "2019-11-07 14:02:43.013946000" 2019.11.07 14:02:43.013 "2019-11-07 14:02:43.013946000"
GOOG 42  100.3 "true"  "2019-11-06 01:45:00.000000000" 2019.11.07 14:02:43.013 "2019-11-07 14:02:43.013946000"


In [22]:
save `:/tmp/allBQSimpleTypes2.csv
system "bq load --autodetect bqkdb.allBQSimpleTypes2 /tmp/allBQSimpleTypes2.csv"

`:/tmp/allBQSimpleTypes2.csv


Waiting on bqjob_r602163027d409e28_0000016e462d8622_1 ... (1s) Current status: DONE   

""
""


In [23]:
system "bq show bqkdb.allBQSimpleTypes2"

"Table ferenc-world:bqkdb.allBQSimpleTypes2"
""
"   Last modified         Schema        Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Lab..
" ----------------- ------------------ ------------ ------------- ------------ ------------------- ------------------ ----..
"  07 Nov 15:03:44   |- s: string       2            110                                                                  ..
"                    |- int: integer                                                                                      ..
"                    |- f: float                                                                                          ..
"                    |- b: boolean                                                                                        ..
"                    |- ts: timestamp                                                                                     ..
"                    |- date: date                                           

## Clean-up

In [24]:
// Cloud Storage
system "gsutil rm ", csfilename

Removing gs://storagebodon/allBQSimpleTypes.csv...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


In [25]:
// local files
system "rm /tmp/allBQSimpleTypes.csv"
system "rm /tmp/allBQSimpleTypes_fixed.csv"
system "rm /tmp/allBQSimpleTypes2.csv"

In [26]:
// BigQuery table
system "bq rm -f bqkdb.allBQSimpleTypes_auto"
system "bq rm -f bqkdb.allBQSimpleTypes2"